In [0]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

In [0]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [0]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [0]:
import textdistance

In [0]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:5]
    variants = [id2word[top] for top in topn]
    
    similarities1 = Counter()
    for word in variants:
        similarities1[word] = metric.normalized_similarity(text, word) 
    
    return similarities1.most_common(1)[0][0]

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
#проверяем качество работы функции

correct = 0
total = 0
mistakes = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        corrected = get_closest_hybrid_match(pair[1], X, vec)
        if corrected == pair[0]:
            correct += 1
        else:
            mistakes.append([pair[0], corrected])
        total += 1      

In [30]:
print(correct/total)  

0.8294047143427886


In [33]:
#ошибки
print(mistakes)

[['симпатичнейшее', 'пластичнейшими'], ['шпионское', 'шпионские'], ['гламурный', 'лагерный'], ['бонда', 'банд'], ['superheadz', 'sueddeutsche'], ['clap', 'place'], ['camera', 'america'], ['получатся', 'ополчатся'], ['язычки', 'язычка'], ['очень', 'чечни'], ['милые', 'милы'], ['насчет', 'защищено'], ['чавеса', 'чавес'], ['попавшим', 'пропавшим'], ['аварийно-спасательных', 'аварийно-восстановительных'], ['в', 'общем'], ['общем', 'как'], ['как', 'вы'], ['вы', 'знаете'], ['знаете', 'из'], ['из', 'моего'], ['моего', 'не'], ['недавнего', 'давнего'], ['пропажу', 'продажу'], ['почте.ру', 'т.п'], ['хорошо', 'хорошее'], ['рите', 'тире'], ['потому', 'трампу'], ['что', 'штат'], ['переждать', 'опережать'], ['дубраве', 'дубрава'], ['люминала', 'людмила'], ['повтыкав', 'аффектов'], ['билетным', 'блатным'], ['кассам', 'классам'], ['что-то', 'что'], ['мощный', 'машинный'], ['нерабочем', 'рабочем'], ['кредиток', 'кредит'], ['хорошо', 'хорошее'], ['капулетти', 'капулети'], ['прислужницу', 'пружину'], ['к